# PYTHON MODULES

In [17]:
import geopandas as gpd
import pandas as pd
from shapely.ops import voronoi_diagram as svd
from shapely.geometry import Polygon, MultiPolygon
from shapely.wkt import loads as load_wkt

# FUNCTIONS

In [2]:
def dropHolesBase(plg):

	'''

	BASIC FUNCTION TO REMOVE / DROP / FILL THE HOLES.

	PARAMETERS:

		plg: plg WHO HAS HOLES / EMPTIES.
			Type: shapely.geometry.MultiPolygon OR shapely.geometry.Polygon

	RETURNS:
		A shapely.geometry.MultiPolygon OR shapely.geometry.Polygon object
	

	'''

	if isinstance(plg, MultiPolygon):

		return MultiPolygon(Polygon(p.exterior) for p in plg)

	elif isinstance(plg, Polygon):

		return Polygon(plg.exterior)


def dropHoles(gdf):

	'''

	REMOVE / DROP / FILL THE HOLES / EMPTIES FOR ITERMS IN GeoDataFrame.
	
	PARAMETERS:
		gdf:
			Type: geopandas.GeoDataFrame

	RETURNS:
		gdf_nohole: GeoDataFrame WITHOUT HOLES
			Type: geopandas.GeoDataFrame
	
	'''

	gdf_nohole = gpd.GeoDataFrame()

	for g in gdf['geometry']:

		geo = gpd.GeoDataFrame(geometry=gpd.GeoSeries(dropHolesBase(g)))

		gdf_nohole=gdf_nohole.append(geo,ignore_index=True)

	gdf_nohole.rename(columns={gdf_nohole.columns[0]:'geometry'}, inplace=True)

	gdf_nohole.crs = gdf.crs

	gdf["geometry"] = gdf_nohole

	return gdf


def thiessen_polygons(gdf, mask):

	'''

	CREATE VORONOI DIAGRAM / THIESSEN POLYGONS:

	PARAMETERS:

		gdf: POINTS / POLYGONS TO BE USED TO CREATE VORONOI DIAGRAM / THIESSEN POLYGONS.
            Type: geopandas.GeoDataFrame

		mask: POLYGON VECTOR USED TO CLIP THE CREATED VORONOI DIAGRAM / THIESSEN POLYGONS.
			Type: GeoDataFrame, GeoSeries, (Multi)Polygon

	RETURNS:

		gdf_vd: THIESSEN POLYGONS
			Type: geopandas.geodataframe.GeoDataFrame
	
	'''
	
	gdf.reset_index(drop=True)

	# CONVERT TO shapely.geometry.MultiPolygon
	smp = gdf.unary_union

	# CREATE PRIMARY VORONOI DIAGRAM BY INVOKING shapely.ops.voronoi_diagram
	poly = load_wkt('POLYGON ((42 24, 64 24, 64 42, 42 42, 42 24))')
	smp_vd = svd(smp, envelope=poly)

	# CONVERT TO GeoSeries AND explode TO SINGLE POLYGONS
	gs = gpd.GeoSeries([smp_vd]).explode()

	# CONVERT TO GEODATAFRAME
	# NOTE THAT IF GDF WAS shapely.geometry.MultiPolygon, IT HAS NO ATTRIBUTE 'crs'
	gdf_vd_primary = gpd.geodataframe.GeoDataFrame(geometry=gs, crs=gdf.crs)
	
	# RESET INDEX
	gdf_vd_primary.reset_index(drop=True)
	
	# SPATIAL JOIN BY INTERSECTING AND DISSOLVE BY `index_right`
	gdf_temp = (gpd.sjoin(gdf_vd_primary, gdf, how='inner', op='intersects').dissolve(by='index_right').reset_index(drop=True))

	gdf_vd = gpd.clip(gdf_temp, mask)

	gdf_vd = dropHoles(gdf_vd)

	return gdf_vd

In [82]:
gdf = gpd.read_file("Data/GeoJson/Yengejeh_Locations_4741.geojson")
gdf.set_crs(4326, allow_override=True)

mask = gpd.read_file("Data/GeoJson/Yengejeh_Aquifer_4741.geojson")
mask.set_crs(4326, allow_override=True)

vd_result = gpd.GeoDataFrame()

for i in range(len(mask)):

    mask_tmp = mask.iloc[[i]]

    gdf['CHECK'] = gdf['geometry'].apply(lambda x: mask_tmp.contains(x))
    gdf_tmp = gdf[gdf['CHECK']].reset_index(drop=True)

    vd = thiessen_polygons(gdf_tmp, mask_tmp)

    vd.set_geometry(col='geometry', inplace=True)
    vd.set_crs(4326, allow_override=True)
    vd["AREA"] = vd.geometry.area * 10000

    vd_result=vd_result.append(vd, ignore_index=True)


vd_result.to_file('output.geojson', driver='GeoJSON')

/home/pooya/w/KHRW-DASHBOARD-LOCAL/Flask/.venv/lib/python3.9/site-packages/geopandas/geodataframe.py:1321: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
<ipython-input-82-eabd41f3f202>:20: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  vd["AREA"] = vd.geometry.area * 10000


In [88]:
gpd.read_file("Data/GeoJson/6008_Sangbast.gdb")

,LAYER,OBJECTID_1,name,Shape_Leng,Area,Shape_Length,Shape_Area,geometry
0,Unknown Area Type,1.0,آبخوان فرهاد گرد- آبکوه,58331.172606,113.374819,0.602535,1.133748e+08,"MULTIPOLYGON (((59.71503 35.76272, 59.71561 35..."
1,Unknown Area Type,2.0,آبخوان سنگ بست,112355.302310,178.983675,1.144977,1.789837e+08,"MULTIPOLYGON (((59.59178 36.01631, 59.59183 36..."


In [89]:
gpd.read_file("Data/GeoJson/4741_yengejeh.gdb")

,LAYER,OBJECTID_1,Shape_Leng,Area,name,Shape_Length,Shape_Area,geometry
0,Unknown Area Type,1.0,45420.978245,98.574907,آبخوان ینگجه,0.462748,9.857491e+07,"MULTIPOLYGON (((58.48390 36.79533, 58.48405 36..."
